# World Happiness Report - Forge Project

Background and motivation: <br> In this project, I was interested in finding insights about how the world's happiness levels changed during the COVID-19 pandemic. The aim of this dataset was to essentially quantify the happiness levels of 140+ countries using various factors such as their GDP per capita, life expectancy, freedom to make life choices, etc. Specifically, I am interested in finding out if there was a significant drop in happiness in 2020, when COVID hit, and if people felt that their freedom to make life choices decreased substantially (due to quarantine, isolation, etc.).

In [83]:
import pandas as pd
import csv
from pymongo import MongoClient
import pymongo
import numpy as np
import seaborn as sns

#### Initial steps: 
1. Read in CSV files <br>
2. Take a look at the datasets, figure out what columns I want to use and their names <br>
3. Turn them into DataFrames for easy manipulation and clean them up wherever needed 

In [54]:
yr19 = pd.read_csv('2019.csv')
yr20 = pd.read_csv('2020.csv')
yr21 = pd.read_csv('2021.csv')

In [95]:
yr19.tail(5)
# Taking a quick look 

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
151,152,Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411
152,153,Tanzania,3.231,0.476,0.885,0.499,0.417,0.276,0.147
153,154,Afghanistan,3.203,0.350,0.517,0.361,0.000,0.158,0.025
154,155,Central African Republic,3.083,0.026,0.000,0.105,0.225,0.235,0.035
155,156,South Sudan,2.853,0.306,0.575,0.295,0.010,0.202,0.091


In [74]:
yr20.head(5)
# Taking a quick look 

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266


In [73]:
yr21.head(5)
# Taking a quick look 

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798


In [64]:
df19 = pd.DataFrame(yr19, columns=['Country or region', 'Score', 'Freedom to make life choices'])
df20 = pd.DataFrame(yr20, columns=['Country name', 'Ladder score', 'Freedom to make life choices'])
df21 = pd.DataFrame(yr21, columns=['Country name', 'Ladder score', 'Freedom to make life choices'])
# Getting only the columns I want and putting them into DataFrames

In [126]:
index19 = pd.Index(range(1, len(df19) + 1, 1))
df19 = df19.set_index(index19)
index20 = pd.Index(range(1, len(df20) + 1, 1))
df20 = df20.set_index(index20)
index21 = pd.Index(range(1, len(df21) + 1, 1))
df21 = df21.set_index(index21)
# Adjusting the indexes

In [122]:
df19.index.name = 'Happiness Rank'
df20.index.name = 'Happiness Rank'
df21.index.name = 'Happiness Rank'
# Changing the index to be the overall rank

In [127]:
df19.tail(3)
# Checking everything is ok

,Country or region,Score,Freedom to make life choices
154,Afghanistan,3.203,0.000
155,Central African Republic,3.083,0.225
156,South Sudan,2.853,0.010


In [128]:
df20.tail(3)
# Checking everything is ok

,Country name,Ladder score,Freedom to make life choices
151,Zimbabwe,3.2992,0.711458
152,South Sudan,2.8166,0.451314
153,Afghanistan,2.5669,0.396573


In [129]:
df21.tail(3)
# Checking everything is ok

,Country name,Ladder score,Freedom to make life choices
147,Rwanda,3.415,0.897
148,Zimbabwe,3.145,0.677
149,Afghanistan,2.523,0.382


Note: the three datasets I picked are not the same lengths- some countries are not included in 2021, for example. For this reason I will drop those countries that are not in all three datasets for the sake of comparison.